---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [113]:
import numpy as np

R = 30                         # 30 years until retirement
T = 60                         # 60 total years
B0 = 100000                    # initial balance is $100,000
D1 = 10000                     # initial savings is $10,000 per year
W1 = 100000                    # withdraw $100,000 first year in retirement
g = 0.02                       # deposit is 2% larger each year
h = 0                          # withdrawals are constant
mn = 0.06                      # earn 6% per month on average
sd = 0.10                      # risk is 10%
numsims = 5000                 # 5,000 simulations

# CALCULATE CASH FLOWS

In [114]:
D = D1 * (1+g)**np.arange(R)
W = W1 * (1+h)**np.arange(T-R)
B0 = np.concatenate(([B0], np.zeros(T)))
D = np.concatenate(([0], D, np.zeros(T-R)))
W = np.concatenate((np.zeros(R+1), W))
CF = B0 + D - W

# SIMULATE FUTURE VALUE FACTORS

Future value of a cash flow $x$ at date $t$ is $$x(1+r_{t+1})(1+r_{t+2})\cdots (1+r_T).$$ fvFactors contains the products of the (1+r) factors for each date $t=0, \ldots, T-1$ and each simulation.

In [115]:
import pandas as pd

rets = np.random.normal(loc=mn, scale=sd, size=(T, numsims))
rets = pd.DataFrame(rets)

def fvs(rets):
    x = np.flip(np.cumprod(1+np.flip(rets, axis=0), axis=0), axis=0)
    return np.concatenate((x, [1]))

fvFactors = rets.apply(fvs)

# CALCULATE ENDING BALANCES

In [116]:
BT = fvFactors.multiply(CF, axis=0).sum()
BT.describe(percentiles=(0.1, 0.25, 0.5, 0.75, 0.9))

count    5.000000e+03
mean     1.097791e+06
std      4.774622e+06
min     -8.718350e+06
10%     -3.178636e+06
25%     -1.872050e+06
50%     -8.362206e+02
75%      2.734108e+06
90%      6.547334e+06
max      5.129224e+07
dtype: float64

# FIGURE 1

In [118]:
import plotly.graph_objects as go

trace = go.Box(x=BT, hovertemplate="%{x}", name="")
fig = go.Figure(trace)
fig.update_layout(
    yaxis_title="",
    xaxis_title="Ending Balance",
    template="plotly_white",
)
fig.show()

# FIGURE 2

In [119]:
grid = [i / 100 for i in range(1, 100)]
pcts = BT.quantile(grid).to_numpy()
trace = go.Scatter(
    x=grid,
    y=pcts,
    mode="lines",
    hovertemplate="%{x:.0%} percentile=%{y:,.0f}<extra></extra>"
)
fig = go.Figure(trace)
fig.update_layout(
    xaxis_title="Percentile",
    yaxis_title="Ending Balance",
    xaxis_tickformat=".0%",
    template="plotly_white",
)
fig.show()